## Forex data gathering and cleaning

In [ ]:
# Imports
import pandas as pd
from pathlib import Path

In [ ]:
# Set file path
file_path = Path("../data/forexdata_raw.csv")

# Create dataframe
forex_df = pd.read_csv(file_path)

In [ ]:
# Get 1 years worth of data (252 trading days)
forex_one_year = forex_df.loc[:251]

In [ ]:
# Write to CSV
forex_one_year.to_csv(f"..//data/forex_one_year_raw.csv", index = False)

In [12]:
# Slice the data into seperate dataframes for USD, JPY, GBP, AUD, RUB, CNY, KRW, HKD, INR
forex_euro_usd = forex_one_year.loc[:,["Date", "USD"]]
forex_euro_jpy = forex_one_year.loc[:,["Date", "JPY"]]
forex_euro_gbp = forex_one_year.loc[:,["Date", "GBP"]]
forex_euro_aud = forex_one_year.loc[:,["Date", "AUD"]]
forex_euro_rub = forex_one_year.loc[:,["Date", "RUB"]]
forex_euro_cny = forex_one_year.loc[:,["Date", "CNY"]]
forex_euro_krw = forex_one_year.loc[:,["Date", "KRW"]]
forex_euro_hkd = forex_one_year.loc[:,["Date", "HKD"]]
forex_euro_inr = forex_one_year.loc[:,["Date", "INR"]]

In [13]:
# Write to CSV
forex_euro_usd.to_csv(f"..//data/forex_euro_usd.csv", index = False)
forex_euro_jpy.to_csv(f"..//data/forex_euro_jpy.csv", index = False)
forex_euro_gbp.to_csv(f"..//data/forex_euro_gbp.csv", index = False)
forex_euro_aud.to_csv(f"..//data/forex_euro_aud.csv", index = False)
forex_euro_rub.to_csv(f"..//data/forex_euro_rub.csv", index = False)
forex_euro_cny.to_csv(f"..//data/forex_euro_cny.csv", index = False)
forex_euro_krw.to_csv(f"..//data/forex_euro_krw.csv", index = False)
forex_euro_hkd.to_csv(f"..//data/forex_euro_hkd.csv", index = False)
forex_euro_inr.to_csv(f"..//data/forex_euro_inr.csv", index = False)

In [15]:
forex_one_year_clean = pd.concat([forex_euro_usd, forex_euro_jpy, forex_euro_gbp, forex_euro_aud, forex_euro_rub, forex_euro_cny, forex_euro_krw, forex_euro_hkd, forex_euro_inr], axis = "columns", join = "inner")

In [22]:
forex_one_year_no_dates = forex_one_year_clean.drop("Date", axis = 1)

In [23]:
forex_one_year_no_dates.head()

,USD,JPY,GBP,AUD,RUB,CNY,KRW,HKD,INR
0,1.1698,122.36,0.90208,1.6563,92.4606,7.8158,1324.20,9.0706,87.1115
1,1.1704,122.09,0.90430,1.6673,93.0481,7.8531,1326.92,9.0735,87.0865
2,1.1727,122.36,0.90662,1.6603,92.3025,7.8845,1332.51,9.0885,86.8040
3,1.1832,123.74,0.90718,1.6585,90.5710,7.9345,1332.89,9.1699,87.1635
4,1.1819,124.14,0.90755,1.6592,90.2935,7.9342,1338.31,9.1598,87.5485


In [24]:
forex_one_year_clean_dates = pd.concat([forex_one_year_no_dates, forex_euro_usd["Date"]], axis = "columns", join = "inner")

In [26]:
forex_one_year_clean_dates = forex_one_year_clean_dates[["Date", "USD", "JPY", "GBP", "AUD", "RUB", "CNY", "KRW", "HKD", "INR"]]

,Date,USD,JPY,GBP,AUD,RUB,CNY,KRW,HKD,INR
0,2020-10-30,1.1698,122.36,0.90208,1.6563,92.4606,7.8158,1324.20,9.0706,87.1115
1,2020-10-29,1.1704,122.09,0.90430,1.6673,93.0481,7.8531,1326.92,9.0735,87.0865
2,2020-10-28,1.1727,122.36,0.90662,1.6603,92.3025,7.8845,1332.51,9.0885,86.8040
3,2020-10-27,1.1832,123.74,0.90718,1.6585,90.5710,7.9345,1332.89,9.1699,87.1635
4,2020-10-26,1.1819,124.14,0.90755,1.6592,90.2935,7.9342,1338.31,9.1598,87.5485


In [28]:
# Write to CSV
forex_one_year_clean_dates.to_csv(f"..//data/forex_one_year_clean.csv", index = False)